In [125]:
import numpy as np
import pandas as pd
from pandas import DataFrame
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
pd.set_option('display.max_rows', None)

### OECD
https://data.oecd.org/
- "Value added in financial and insurance activities": "DV_VA%" (% of GDP) & "DV_VA" (* OECD_gdp)
- "General government net lending": "IV_lending%" (% of GDP) & "IV_lending" (* OECD_gdp) 
- "Real effective exchange rates": "C_REER" (index, 2010=100)
- "CPI: all items": "C_cpi" (growth rate)
- "Gross domestic product (output approach)" (constant prices, constant exchange rates, OECD base year): "C_gdp"

### WB
https://databank.worldbank.org/home.aspx
- "Foreign direct investment, net outflows (% of GDP)": "IV_fdi_outflow%"
- "Foreign direct investment, net (BoP, current US$)" (net outflow - net inflow): "fdi_net"

    Foreign direct investment are the net inflows of investment to acquire a lasting management interest (10 percent or more of voting stock) in an enterprise operating in an economy other than that of the investor. It is the sum of equity capital, reinvestment of earnings, other long-term capital, and short-term capital as shown in the balance of payments. This series shows total net FDI. In BPM6, financial account balances are calculated as the change in assets minus the change in liabilities. Net FDI outflows are assets and net FDI inflows are liabilities. Data are in current U.S. dollars.


- "Net trade in goods and services (BoP, current US$)": "IV_trade_balance"
- "Broad money (% of GDP)": "C_M2"

### GDD 
https://www.imf.org/external/datamapper/datasets/GDD
- "nfc_ls"(nonfinancial corporate debt, loans and debt securities, % of GDP): "DV_nfc_ls%" & "DV_nfc_ls" (* IMF_gdp)
- "hh_ls"(houshold debt, loans and debt securities, % of GDP): "DV_hh_ls%" & "DV_hh_ls" (* IMF_gdp)

### Historical Puclic Finance 
https://www.imf.org/external/np/fad/histdb/index.htm
- "exp"(government expenditure, % of GDP): "IV_gov_exp%" & "IV_gov_exp" (* IMF_gdp) 

### WID
https://wid.world/data/
- "gini": "IV_gini"

### WIID
https://www.wider.unu.edu/database/wiid
- "gini": "IV_wgini"
- "gdp": "C_wgdp"

#### OECD Database Import

In [126]:
# import OECD.csv and select features
oecd = pd.read_csv('../data_source/OECD.csv')

In [127]:
# database cleaning
def oecd_clean(df):
    
    # select columns
    df = df[['Subject','Country','Year','Value']]
    # select subjects
    df = df.loc[(df['Subject']=='Value added in financial and insurance activities')
                |(df['Subject']=='General government net lending')
                |(df['Subject']=='Real effective exchange rates')
                |(df['Subject']=='CPI: all items')
                |(df['Subject']=='Real value added in financial and insurance activities')]
    
    # replace country names to a common form
    country_rep = {"China (People's Republic of)": "China"}
    for rep in country_rep:
        df.Country = df.Country.replace(rep, country_rep[rep])
        
    return df

oecd = oecd_clean(oecd)

In [128]:
# import gdp.csv/OECD
gdp = pd.read_csv('../data_source/gdp.csv')

In [129]:
# database cleaning
def gdp_clean(df):
    
    # select measure: constant prices, constant exchange rates, OECD base year
    df = df.loc[df['MEASURE']=='VXVOB']
    
    # rename columns
    df = df[['Transaction','Country','Year','Value']]
    # rename column
    df = df.rename(columns={'Transaction':'Subject'})
    
    # delete subject that is not a country
    sub_del = ['European Union (28 countries)', 'OECD - Total', 'Euro area (19 countries)']
    df = df[~df.Country.isin(sub_del)]
    
    # replace country names to a common form
    country_rep = {"China (People's Republic of)": "China"}
    for rep in country_rep:
        df.Country = df.Country.replace(rep, country_rep[rep])
        
    return df

gdp = gdp_clean(gdp)

#### World Bank Database Import

In [130]:
# database cleaning
def wb_clean(df):
    # drop unnecessary columns
    df = df.drop(['Unnamed: 0', 'Country Code', 'Indicator Code', 'V65'], axis=1)
    # change from wide to long format
    df = df.pivot_table(index='Country Name', columns='Indicator Name').unstack().reset_index()
    # rename columns
    df = df.rename(columns={'level_0':'Year', 'Indicator Name':'Subject',
                                       'Country Name':'Country', 0:'Value'})
    convert_dict = {'Year': int, 
                'Value': float} 
  
    # delete subject that is not a country
    sub_del = ['Arab World', 'Caribbean small states', 'Central Europe and the Baltics', 'Early-demographic dividend', 
           'East Asia & Pacific', 'East Asia & Pacific (IDA & IBRD countries)', 'East Asia & Pacific (excluding high income)',
           'Euro area', 'Europe & Central Asia', 'Europe & Central Asia (IDA & IBRD countries)',
           'Europe & Central Asia (excluding high income)', 'European Union', 'Fragile and conflict affected situations',
           'Heavily indebted poor countries (HIPC)', 'High income', 'IBRD only', 'IDA & IBRD total', 'IDA blend',
           'IDA only', 'IDA total', 'Late-demographic dividend', 'Latin America & Caribbean',
           'Latin America & Caribbean (excluding high income)', 'Latin America & the Caribbean (IDA & IBRD countries)',
           'Least developed countries: UN classification', 'Low & middle income',
           'Low income', 'Lower middle income', 'Middle East & North Africa', 
           'Middle East & North Africa (excluding high income)', 'Middle East & North Africa (IDA & IBRD countries)', 
           'Middle income', 'North America', 'OECD - Total', 'OECD members', 'Other small states', 'Pacific island small states',
           'Post-demographic dividend', 'Pre-demographic dividend', 'Small states', 'South Africa', 'South Asia',
           'South Asia (IDA & IBRD)', 'Sub-Saharan Africa', 'Sub-Saharan Africa (excluding high income)',
           'Sub-Saharan Africa (IDA & IBRD countries)', 'Upper middle income', 'World']

    df = df[~df.Country.isin(sub_del)]
    
    df = df.astype(convert_dict)
    # replace country names to a common form
    country_rep = {"Russian Federation" : "Russia",
                   "Korea, Rep." : "Korea",
                   "Micronesia, Fed. Sts.": "Micronesia",
                   "Timor-Leste": "Timor Leste",
                   "Venezuela, RB": "Venezuela",
                   "Yemen, Rep.": "Yemen",
                   "Syrian Arab Republic": "Syria",
                   "Iran, Islamic Rep.": "Iran",
                   "Hong Kong SAR, China": "Hong Kong SAR",
                   "Macao SAR, China": "Macao SAR",
                   "China (People's Republic of)": "China",
                   "Brunei Darussalam": "Brunei",
                   "Lao PDR": "Laos",
                   "Egypt, Arab Rep.": "Egypt"}
    for rep in country_rep:
        df.Country = df.Country.replace(rep, country_rep[rep])
        
    return df

In [131]:
#fdi_outflow = pd.read_csv('../data_source/outflow.csv')
#fdi_outflow = wb_clean(fdi_outflow)

trade_balance = pd.read_csv('../data_source/netgoodservices.csv')
trade_balance = wb_clean(trade_balance)

#m2 = pd.read_csv('../data_source/broad_money.csv')
#m2 = wb_clean(m2)

In [132]:
# database cleaning
def fdi_net_clean(df):
    # drop unnecessary columns
    df = df.drop(['Country Code', 'Indicator Code'], axis=1)
    # change from wide to long format
    df = df.pivot_table(index='Country Name', columns='Indicator Name').unstack().reset_index()
    # rename columns
    df = df.rename(columns={'level_0':'Year', 'Indicator Name':'Subject',
                                       'Country Name':'Country', 0:'Value'})
    convert_dict = {'Year': int, 
                'Value': float} 
  
    # delete subject that is not a country
    sub_del = ['Arab World', 'Caribbean small states', 'Central Europe and the Baltics', 'Early-demographic dividend', 
           'East Asia & Pacific', 'East Asia & Pacific (IDA & IBRD countries)', 'East Asia & Pacific (excluding high income)',
           'Euro area', 'Europe & Central Asia', 'Europe & Central Asia (IDA & IBRD countries)',
           'Europe & Central Asia (excluding high income)', 'European Union', 'Fragile and conflict affected situations',
           'Heavily indebted poor countries (HIPC)', 'High income', 'IBRD only', 'IDA & IBRD total', 'IDA blend',
           'IDA only', 'IDA total', 'Late-demographic dividend', 'Latin America & Caribbean',
           'Latin America & Caribbean (excluding high income)', 'Latin America & the Caribbean (IDA & IBRD countries)',
           'Least developed countries: UN classification', 'Low & middle income',
           'Low income', 'Lower middle income', 'Middle East & North Africa', 
           'Middle East & North Africa (excluding high income)', 'Middle East & North Africa (IDA & IBRD countries)', 
           'Middle income', 'North America', 'OECD - Total', 'OECD members', 'Other small states', 'Pacific island small states',
           'Post-demographic dividend', 'Pre-demographic dividend', 'Small states', 'South Africa', 'South Asia',
           'South Asia (IDA & IBRD)', 'Sub-Saharan Africa', 'Sub-Saharan Africa (excluding high income)',
           'Sub-Saharan Africa (IDA & IBRD countries)', 'Upper middle income', 'World']

    df = df[~df.Country.isin(sub_del)]
    
    df = df.astype(convert_dict)
    # replace country names to a common form
    country_rep = {"Russian Federation" : "Russia",
                   "Korea, Rep." : "Korea",
                   "Micronesia, Fed. Sts.": "Micronesia",
                   "Timor-Leste": "Timor Leste",
                   "Venezuela, RB": "Venezuela",
                   "Yemen, Rep.": "Yemen",
                   "Syrian Arab Republic": "Syria",
                   "Iran, Islamic Rep.": "Iran",
                   "Hong Kong SAR, China": "Hong Kong SAR",
                   "Macao SAR, China": "Macao SAR",
                   "China (People's Republic of)": "China",
                   "Brunei Darussalam": "Brunei",
                   "Lao PDR": "Laos",
                   "Egypt, Arab Rep.": "Egypt"}
    for rep in country_rep:
        df.Country = df.Country.replace(rep, country_rep[rep])
        
    return df

In [133]:
fdi_net = pd.read_csv('../data_source/fdi_net.csv')
fdi_net = fdi_net_clean(fdi_net)

#### IMF GDD Database Import

In [134]:
gdd = pd.read_csv('../data_source/global_debt_database.csv')

# database cleaning
def gdd_clean(df):
    # select columns
    df = df[['country', 'year', 'nfc_ls', 'hh_ls', 'ngdp']]
    # rename columns
    df = df.rename(columns={'year':'Year', 'country':'Country'})
    # change country name
    country_rep = {"Russian Federation" : "Russia",
                   "Korea, Republic of" : "Korea",
                   "Micronesia, Fed. Sts.": "Micronesia",
                   "Timor-Leste": "Timor Leste",
                   "Venezuela, RB": "Venezuela",
                   "Yemen, Rep.": "Yemen",
                   "Syrian Arab Republic": "Syria",
                   "Iran, Islamic Rep.": "Iran",
                   "Hong Kong SAR, China": "Hong Kong SAR",
                   "Macao SAR, China": "Macao SAR",
                   "China, Mainland": "China",
                   "Brunei Darussalam": "Brunei",
                   "Iran, I. Rep. Of": "Iran",
                   "Marshall Islands, Rep.": "Marshall Islands",
                   "São Tomé and Príncipe": "Sao Tome and Principe",
                   "U.A.E.": "United Arab Emirates",
                   "Venezuela, Republica Bolivariana de": "Venezuela",
                   "Antigua & Barbuda": "Antigua and Barbuda",
                   "Bosnia & Herzegovina": "Bosnia and Herzegovina",
                   "Trinidad & Tobago": "Trinidad and Tobago",
                   "Congo, Dem. Rep. of": "Congo, Dem. Rep.",
                   "Congo, Republic of": "Congo, Rep.",
                   "Cote D'Ivoire": "Cote d'Ivoire",
                   "Taiwan Province of China": "Taiwan",
                   "C.A.R.": "Central African Republic"
                  }
    for rep in country_rep:
        df.Country = df.Country.replace(rep, country_rep[rep])
    
    # replace 0 with NAs
    df.replace(0, np.nan, inplace=True)
    
    return df

gdd = gdd_clean(gdd)

#### Public Finance Database Import

In [135]:
public_finance = pd.read_csv('../data_source/historical_public_finance_dataset.csv')

# clean public_finance dataframe
def public_finance_clean(df):
    # select columns
    df = df[['country', 'year', 'exp']]
    # rename columns
    df = df.rename(columns={'year':'Year', 'country':'Country'})
    
    # change country name
    country_rep = {"Russian Federation" : "Russia",
                   "South Korea" : "Korea",
                   "Hong Kong": "Hong Kong SAR"
                     }
    for rep in country_rep:
        df.Country = df.Country.replace(rep, country_rep[rep])
    
    # replace 0 with NAs
    df.replace(0, np.nan, inplace=True)
    
    return df

public_finance = public_finance_clean(public_finance)

#### Gini Database Import

#### WIID Database Import

In [136]:
wiid = pd.read_csv('../data_source/WIID_31MAR2021.csv')

In [137]:
def gdp_wiid_clean(df):
    # select columns
    df = df[['country', 'year', 'gdp']]
    # drop duplicates
    df = df.drop_duplicates(['year', 'gdp'])
    # drop inconsistent rows within the same year
    df = df.drop(df.index[df.gdp==19941][0],axis=0)
    df = df.drop(df.index[df.gdp==28206][0],axis=0)
    
    # rename columns
    df = df.rename(columns={'year':'Year', 'country':'Country', 'gdp': 'gdp_wiid'})
    
    # change country name
    country_rep = {"China (People's Republic of)": "China",
                   "Korea, Republic of" : "Korea",
                   "Congo, Democratic Republic of the": "Congo, Dem. Rep.",
                   "Congo, Republic of the": "Congo, Rep.",
                   "Slovakia": "Slovak Republic",
                   "Timor-Leste": "Timor Leste",
                   "Hong Kong": "Hong Kong SAR",
                   "Kyrgyzstan": "Kyrgyz Republic",
                   "Micronesia, Federated States of": "Micronesia",
                   "Czechia": "Czech Republic"
                     }
    for rep in country_rep:
        df.Country = df.Country.replace(rep, country_rep[rep])

    return df

gdp_wiid = gdp_wiid_clean(wiid)

#### Combine Databases

In [138]:
# combine the above databases
oecd = oecd.append([gdp, trade_balance, fdi_net]) # fdi_outflow, m2
oecd = oecd.sort_values(by=['Country', 'Subject', 'Year'])
oecd = oecd.pivot_table(values='Value', index=['Country', 'Year'], columns=['Subject']).reset_index()

oecd = pd.merge(oecd, gdd, on=['Country', 'Year'], how='outer')
oecd = pd.merge(oecd, public_finance, on=['Country', 'Year'], how='outer')
#oecd = pd.merge(oecd, gini, on=['Country', 'Year'], how='outer')
oecd = pd.merge(oecd, gdp_wiid, on=['Country', 'Year'], how='outer')
#oecd = pd.merge(oecd, wgini, on=['Country', 'Year'], how='outer')

oecd = oecd.sort_values(by=['Country', 'Year'])

In [139]:
oecd = oecd.rename(columns={
    "Value added in financial and insurance activities": "DV_VA%",
    "Real value added in financial and insurance activities": "DV_RVA_pc",
    "nfc_ls": "DV_nfc_ls%",
    "hh_ls": "DV_hh_ls%",
    "General government net lending": "IV_lending%",
    "exp": "IV_gov_exp%",
    #"gini": "IV_gini",
    "Net trade in goods and services (BoP, current US$)": "IV_trade_balance",
    #"Foreign direct investment, net outflows (% of GDP)": "IV_fdi_outflow%",
    "Real effective exchange rates": "C_REER",
    "CPI: all items": "C_cpi",
    "Gross domestic product (output approach)": "OECD_gdp",
    "ngdp":"IMF_gdp",
    #"Broad money (% of GDP)": "C_M2",
    "gdp_wiid": "C_wgdp",
    #"gini_wiid": "IV_wgini",
    "Foreign direct investment, net (BoP, current US$)": "IV_fdi_net"
})

In [140]:
oecd.isnull().sum()

Country             0    
Year                0    
C_cpi               19896
IV_fdi_net          15390
IV_lending%         21045
OECD_gdp            20042
IV_trade_balance    15524
C_REER              20366
DV_RVA_pc           20981
DV_VA%              20878
DV_nfc_ls%          19874
DV_hh_ls%           19859
IMF_gdp             11189
IV_gov_exp%         16565
C_wgdp              18407
dtype: int64

In [141]:
# get log of C_gdp and IV_trade_balance
oecd['DV_VA'] = oecd['DV_VA%'] * oecd['OECD_gdp']/100
oecd['IV_lending'] = oecd['IV_lending%'] * oecd['OECD_gdp']/100
oecd['DV_nfc_ls'] = oecd['DV_nfc_ls%'] * oecd['IMF_gdp']/100
oecd['DV_hh_ls'] = oecd['DV_hh_ls%'] * oecd['IMF_gdp']/100
oecd['IV_gov_exp'] = oecd['IV_gov_exp%'] * oecd['IMF_gdp']/100

#oecd['C_gdp_log'] = np.log(oecd['C_gdp'])
oecd['C_wgdp_log'] = np.log(oecd['C_wgdp'])
oecd['IV_trade_balance_log'] = np.sign(oecd['IV_trade_balance'])*np.log(abs(oecd['IV_trade_balance']))
oecd['IV_fdi_net_log'] = np.sign(oecd['IV_fdi_net'])*np.log(abs(oecd['IV_fdi_net']))

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [142]:
# get lag 1 period of DV
# DV with percentage of GDP
oecd['DV_VA%_lag1'] = oecd['DV_VA%'].shift(1)
oecd['DV_nfc_ls%_lag1'] = oecd['DV_nfc_ls%'].shift(1)
oecd['DV_hh_ls%_lag1'] = oecd['DV_hh_ls%'].shift(1)

# DV with multiple of GDP
oecd['DV_VA_lag1'] = oecd['DV_VA'].shift(1)
oecd['DV_nfc_ls_lag1'] = oecd['DV_nfc_ls'].shift(1)
oecd['DV_hh_ls_lag1'] = oecd['DV_hh_ls'].shift(1)

oecd['DV_RVA_pc_lag1'] = oecd['DV_RVA_pc'].shift(1)

In [143]:
# get lag 1 period and 2 periods of IV
# IV with percentage of GDP
oecd['IV_lending%_lag1'] = oecd['IV_lending%'].shift(1)
oecd['IV_gov_exp%_lag1'] = oecd['IV_gov_exp%'].shift(1)

oecd['IV_lending%_lag2'] = oecd['IV_lending%'].shift(2)
oecd['IV_gov_exp%_lag2'] = oecd['IV_gov_exp%'].shift(2)

# IV with multiple of GDP
oecd['IV_lending_lag1'] = oecd['IV_lending'].shift(1)
oecd['IV_gov_exp_lag1'] = oecd['IV_gov_exp'].shift(1)
#oecd['IV_gini_lag1'] = oecd['IV_gini'].shift(1)
#oecd['IV_wgini_lag1'] = oecd['IV_wgini'].shift(1)
oecd['IV_trade_balance_log_lag1'] = oecd['IV_trade_balance_log'].shift(1)
#oecd['IV_fdi_outflow_lag1'] = oecd['IV_fdi_outflow'].shift(1)
oecd['IV_fdi_net_lag1'] = oecd['IV_fdi_net'].shift(1)

oecd['IV_lending_lag2'] = oecd['IV_lending'].shift(2)
oecd['IV_gov_exp_lag2'] = oecd['IV_gov_exp'].shift(2)
#oecd['IV_gini_lag2'] = oecd['IV_gini'].shift(2)
#oecd['IV_wgini_lag2'] = oecd['IV_wgini'].shift(2)
oecd['IV_trade_balance_log_lag2'] = oecd['IV_trade_balance_log'].shift(2)
#oecd['IV_fdi_outflow_lag2'] = oecd['IV_fdi_outflow'].shift(2)
oecd['IV_fdi_net_lag2'] = oecd['IV_fdi_net'].shift(2)

In [144]:
# get DV's percentage change 
# DV with percentage of GDP
oecd['DV_VA%_pc'] = oecd['DV_VA%'].pct_change()
oecd['DV_nfc_ls%_pc'] = oecd['DV_nfc_ls%'].pct_change()
oecd['DV_hh_ls%_pc'] = oecd['DV_hh_ls%'].pct_change()

# DV with multiple of GDP
oecd['DV_VA_pc'] = oecd['DV_VA'].pct_change()
oecd['DV_nfc_ls_pc'] = oecd['DV_nfc_ls'].pct_change()
oecd['DV_hh_ls_pc'] = oecd['DV_hh_ls'].pct_change()

# get lag 1 period of DV_pc
# DV with percentage of GDP
oecd['DV_VA%_pc_lag1'] = oecd['DV_VA%_pc'].shift(1)
oecd['DV_nfc_ls%_pc_lag1'] = oecd['DV_nfc_ls%_pc'].shift(1)
oecd['DV_hh_ls%_pc_lag1'] = oecd['DV_hh_ls%_pc'].shift(1)

# DV with multiple of GDP
oecd['DV_VA_pc_lag1'] = oecd['DV_VA_pc'].shift(1)
oecd['DV_nfc_ls_pc_lag1'] = oecd['DV_nfc_ls_pc'].shift(1)
oecd['DV_hh_ls_pc_lag1'] = oecd['DV_hh_ls_pc'].shift(1)

In [145]:
# get IV's percentage change
# IV with percentage of GDP
oecd['IV_lending%_pc'] = oecd['IV_lending%'].pct_change()
oecd['IV_gov_exp%_pc'] = oecd['IV_gov_exp%'].pct_change()

# IV with multiple of GDP
oecd['IV_lending_pc'] = oecd['IV_lending'].pct_change()
oecd['IV_gov_exp_pc'] = oecd['IV_gov_exp'].pct_change()
oecd['IV_trade_balance_pc'] = oecd['IV_trade_balance'].pct_change()
#oecd['IV_fdi_net_pc'] = oecd['IV_fdi_net'].pct_change()

# get lag 1 period and 2 periods of IV_pc
# IV with percentage of GDP
oecd['IV_lending%_pc_lag1'] = oecd['IV_lending%_pc'].shift(1)
oecd['IV_gov_exp%_pc_lag1'] = oecd['IV_gov_exp%_pc'].shift(1)

oecd['IV_lending%_pc_lag2'] = oecd['IV_lending%_pc'].shift(2)
oecd['IV_gov_exp%_pc_lag2'] = oecd['IV_gov_exp%_pc'].shift(2)

# IV with multiple of GDP
oecd['IV_lending_pc_lag1'] = oecd['IV_lending_pc'].shift(1)
oecd['IV_gov_exp_pc_lag1'] = oecd['IV_gov_exp_pc'].shift(1)
oecd['IV_trade_balance_pc_lag1'] = oecd['IV_trade_balance_pc'].shift(1)
#oecd['IV_fdi_net_pc_lag1'] = oecd['IV_fdi_net_pc'].shift(1)

oecd['IV_lending_pc_lag2'] = oecd['IV_lending_pc'].shift(2)
oecd['IV_gov_exp_pc_lag2'] = oecd['IV_gov_exp_pc'].shift(2)
oecd['IV_trade_balance_pc_lag2'] = oecd['IV_trade_balance_pc'].shift(2)
#oecd['IV_fdi_net_pc_lag2'] = oecd['IV_fdi_net_pc'].shift(2)

In [146]:
# get CV's percentage change
#oecd['C_NFLplus'] = oecd['C_NFL'].pct_change()
#oecd['C_REERplus'] = oecd['C_REER'].pct_change()
#oecd['C_cpiplus'] = oecd['C_cpi'].pct_change()
#oecd['C_gdpplus'] = oecd['C_gdp'].pct_change()
oecd['C_wgdpplus'] = oecd['C_wgdp'].pct_change()
#oecd['C_m2plus'] = oecd['C_M2'].pct_change()

In [147]:
oecd.to_csv('../data_processing/financialization_df.csv')

In [148]:
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)
oecd.describe()

<ipython-input-148-5eae15464d34>:3: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


,Year,C_cpi,IV_fdi_net,IV_lending%,OECD_gdp,IV_trade_balance,C_REER,DV_RVA_pc,DV_VA%,DV_nfc_ls%,DV_hh_ls%,IMF_gdp,IV_gov_exp%,C_wgdp,DV_VA,IV_lending,DV_nfc_ls,DV_hh_ls,IV_gov_exp,C_wgdp_log,IV_trade_balance_log,IV_fdi_net_log,DV_VA%_lag1,DV_nfc_ls%_lag1,DV_hh_ls%_lag1,DV_VA_lag1,DV_nfc_ls_lag1,DV_hh_ls_lag1,DV_RVA_pc_lag1,IV_lending%_lag1,IV_gov_exp%_lag1,IV_lending%_lag2,IV_gov_exp%_lag2,IV_lending_lag1,IV_gov_exp_lag1,IV_trade_balance_log_lag1,IV_fdi_net_lag1,IV_lending_lag2,IV_gov_exp_lag2,IV_trade_balance_log_lag2,IV_fdi_net_lag2,DV_VA%_pc,DV_nfc_ls%_pc,DV_hh_ls%_pc,DV_VA_pc,DV_nfc_ls_pc,DV_hh_ls_pc,DV_VA%_pc_lag1,DV_nfc_ls%_pc_lag1,DV_hh_ls%_pc_lag1,DV_VA_pc_lag1,DV_nfc_ls_pc_lag1,DV_hh_ls_pc_lag1,IV_lending%_pc,IV_gov_exp%_pc,IV_lending_pc,IV_gov_exp_pc,IV_trade_balance_pc,IV_lending%_pc_lag1,IV_gov_exp%_pc_lag1,IV_lending%_pc_lag2,IV_gov_exp%_pc_lag2,IV_lending_pc_lag1,IV_gov_exp_pc_lag1,IV_trade_balance_pc_lag1,IV_lending_pc_lag2,IV_gov_exp_pc_lag2,IV_trade_balance_pc_lag2,C_wgdpplus
count,22045.000000,2149.000000,6.655000e+03,1000.000000,2.003000e+03,6.521000e+03,1679.000000,1064.000000,1167.000000,2171.000000,2186.000000,1.085600e+04,5480.000000,3638.000000,1.156000e+03,9.820000e+02,2.171000e+03,2.186000e+03,3.083000e+03,3638.000000,6521.000000,6235.000000,1167.000000,2171.000000,2186.000000,1.156000e+03,2.171000e+03,2.186000e+03,1064.000000,1000.000000,5480.000000,1000.000000,5480.000000,9.820000e+02,3.083000e+03,6521.000000,6.655000e+03,9.820000e+02,3.083000e+03,6520.000000,6.654000e+03,21178.000000,21988.000000,21988.000000,21178.000000,21988.000000,21988.000000,21177.000000,21987.000000,21987.000000,21177.000000,21987.000000,21987.000000,21197.000000,21627.000000,21197.000000,2.154100e+04,22015.000000,21196.000000,21626.000000,21195.000000,21625.000000,21196.000000,2.154000e+04,22014.000000,21195.000000,2.153900e+04,22013.000000,21986.000000
mean,1942.761805,17.176366,-4.242334e+08,-2.103180,8.832779e+05,6.463522e+08,113.516314,3.838199,5.759766,65.066879,36.104242,3.623969e+04,20.224061,19684.615448,6.131236e+04,-4.631550e+04,3.895588e+04,2.563508e+04,9.808108e+03,9.455754,-7.237318,-11.941281,5.759766,65.066879,36.104242,6.131236e+04,3.895588e+04,2.563508e+04,3.838199,-2.103180,20.224061,-2.103180,20.224061,-4.631550e+04,9.808108e+03,-7.237318,-4.242334e+08,-4.631550e+04,9.808108e+03,-7.235155,-4.242600e+08,0.000896,0.034912,0.007504,0.003079,0.147898,0.108264,0.000896,0.034914,0.007504,0.003079,0.147905,0.108269,-0.045748,0.006199,-0.029070,1.013394e+05,0.071406,-0.045750,0.006199,-0.045752,0.006199,-0.029071,1.013441e+05,0.071409,-0.029072,1.013488e+05,0.071422,0.010678
std,61.930723,111.786620,1.659494e+10,4.066963,2.026634e+06,3.723059e+10,109.867716,8.020146,3.669052,51.404398,29.266453,4.949716e+05,14.209101,17380.966716,1.606355e+05,1.709362e+05,2.121195e+05,1.583150e+05,7.457976e+04,1.031558,19.388576,15.458466,3.669052,51.404398,29.266453,1.606355e+05,2.121195e+05,1.583150e+05,8.020146,4.066963,14.209101,4.066963,14.209101,1.709362e+05,7.457976e+04,19.388576,1.659494e+10,1.709362e+05,7.457976e+04,19.389276,1.659619e+10,0.078370,3.605304,0.181478,0.104601,10.461219,9.297148,0.078371,3.605386,0.181482,0.104604,10.461457,9.297360,4.713839,0.148564,2.395426,1.487336e+07,27.667067,4.713950,0.148567,4.714062,0.148571,2.395483,1.487370e+07,27.667695,2.395539,1.487405e+07,27.668324,0.237650
min,1800.000000,-7.633931,-4.127800e+11,-32.061638,6.644201e+03,-7.617150e+11,31.343123,-40.685919,1.504824,0.067251,0.084322,1.028898e-13,0.684444,385.000000,2.874091e+02,-2.091968e+06,1.215700e-02,1.500000e-04,8.953602e-15,5.953243,-27.358838,-26.746181,1.504824,0.067251,0.084322,2.874091e+02,1.215700e-02,1.500000e-04,-40.685919,-32.061638,0.684444,-32.061638,0.684444,-2.091968e+06,8.953602e-15,-27.358838,-4.127800e+11,-2.091968e+06,8.953602e-15,-27.358838,-4.127800e+11,-0.832127,-0.999245,-0.995387,-0.994915,-1.000000,-1.000000,-0.832127,-0.999245,-0.995387,-0.994915,-1.000000,-1.000000,-658.657206,-0.974326,-280.331546

In [149]:
oecd.describe().to_csv('../data_processing/variable_describe.csv')

#### Plot IV_fdi_net

In [37]:
fdi_ls = ['United States', 'United Kingdom', 'China', 'France', 'Germany', 'Japan', 'Korea', 'Netherlands', 
          'Norway', 'Canada', 'Switzerland', 'Swedan', 'Belgium']
fdi_df = oecd[oecd.Country.isin(fdi_ls)]
fdi_df = fdi_df[['Country', 'Year', 'IV_fdi_net']]

In [39]:
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot
import plotly.graph_objs as go
import cufflinks as cf
init_notebook_mode(connected=True)
cf.go_offline()

In [40]:
oecd['Year'] = pd.to_datetime(oecd['Year'], format='%Y')
df = oecd.set_index('Year', append=True)
oecd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22167 entries, 10404 to 9343
Data columns (total 55 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Country                   22167 non-null  object        
 1   Year                      22167 non-null  datetime64[ns]
 2   C_M2                      7260 non-null   float64       
 3   C_cpi                     2151 non-null   float64       
 4   IV_fdi_net                6657 non-null   float64       
 5   IV_fdi_outflow            7732 non-null   float64       
 6   IV_lending                982 non-null    float64       
 7   OECD_gdp                  2005 non-null   float64       
 8   IV_trade_balance          6523 non-null   float64       
 9   C_REER                    1681 non-null   float64       
 10  DV_VA                     1156 non-null   float64       
 11  DV_nfc_ls                 2173 non-null   float64       
 12  DV_hh_ls       

In [41]:
pd.options.plotting.backend = "plotly"
fdi_df.plot(x='Year', y='IV_fdi_net', color='Country', kind='scatter')